# Clase 21: Compilación y Paralelismo

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

**Profesor: Pablo Badilla**

## Tipos de Lenguaje

<center>
<img src='./resources/tipos_lenguajes.png' width=800 />
</center>


---

## Compiladores

<center>
<img src='./resources/numba.png' width=600/>
</center>

Un proyecto interesante la librería **`Numba`** **analiza y compila funciones de Python**. Compiladores como Numba, diseñados para compilar código en ejecución (y no previo a la ejecución) se denomina compiladores **JIT** (just in time). 

Numba permite compilar funciones individuales de Python usado una *máquina virtual de bajo nivel* o LLVM por sus siglas en inglés. LLVM es un conjunto de herramientas pensadas para escribir compiladores y no depende del lenguaje utilizado para programar. 

Por medio de LLVM Numba inspecciona funciones de Python y las compila utilizando una capa de representación intermedia similar a código *assembly*. La potencia de esta inspección radica en la inferencia de tipos de datos generando una versiones compiladas con tipos de datos estáticos.

Numba se basa principalmente en el decorador `@jit` con el cual se definen las funciones a compilar.

**Ejemplo: Calcular el valor de $\pi$ usando Montecarlo**


Idea: 

<center>
<img src='./resources/montecarlo.png' />
<center/>
    
$$\frac{\text{area círculo}}{\text{area cuadrado}} = \frac{\pi r^2}{(2r)^2} $$

$$ 4* \frac{\text{area círculo}}{\text{area cuadrado}} = \pi $$


Y después simulamos que lanzamos puntos al azar a nuestra figura y contamos: 

$$ 4* \frac{\text{puntos en el circulo}}{\text{puntos en el cuadrado}} = \pi $$





Para comprobar el aumento de rendimiento de la compilación, usaremos 3 implementaciones distintas:
    
    1. Python.
    2. Numpy.
    3. Python con Numba.

### $\pi$ con Montecarlo en `Python`

In [1]:
import random 

def monte_carlo_pi_python(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()

        if (x ** 2 + y ** 2) < 1.0:
            acc += 1

    return 4.0 * acc / nsamples

In [45]:
monte_carlo_pi_python(10000000)

3.141094

In [46]:
%timeit monte_carlo_pi_python(100000)

42.2 ms ± 2.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### $\pi$ con Montecarlo en `Numpy`

In [47]:
import numpy as np


def monte_carlo_pi_numpy(nsamples):
    acc = 0
    x = np.random.rand(nsamples)
    y = np.random.rand(nsamples)

    op = x ** 2 + y ** 2
    dentro_circulo = op[op < 1.0]

    return 4.0 * np.count_nonzero(dentro_circulo) / nsamples

In [48]:
monte_carlo_pi_numpy(100000)

3.13412

In [49]:
%timeit monte_carlo_pi_numpy(100000)

2.84 ms ± 275 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [50]:
42.2 / 2.84

14.859154929577466

### $\pi$ con Montecarlo en `Numba`

Y ahora probamos con una función compilada usando el decorador `@jit`.

In [51]:
import random

from numba import jit


@jit(nopython=True)
def monte_carlo_pi_numba(nsamples):
    acc = 0
    for i in range(nsamples):
        x = random.random()
        y = random.random()

        if (x ** 2 + y ** 2) < 1.0:
            acc += 1

    return 4.0 * acc / nsamples

In [52]:
monte_carlo_pi_numba(100000)

3.14372

In [53]:
%timeit monte_carlo_pi_numba(100000)

1.21 ms ± 29.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Numba y Numpy

`Numba` también está diseñado para funcionar en conjunto con `numpy`

In [55]:
@jit(nopython=True)
def monte_carlo_pi_numpy_numba(nsamples):
    acc = 0
    x = np.random.rand(nsamples)
    y = np.random.rand(nsamples)

    op = x ** 2 + y ** 2
    dentro_circulo = op[op < 1.0]

    return 4.0 * np.count_nonzero(dentro_circulo) / nsamples

In [56]:
monte_carlo_pi_numpy_numba(100000)

3.14536

In [57]:
%timeit monte_carlo_pi_numpy_numba(100000)

1.73 ms ± 70.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Importante: `Numba` solo compila código de Python y `Numpy`

Está en general diseñado para optimizar tareas matemáticas y con ciclos.
No entiende librerías más complejas como `pandas` por ejemplo.





In [58]:
import pandas as pd

x = {"a": [1, 2, 3], "b": [20, 30, 40]}

In [60]:
def use_pandas(a):  # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a)  # Numba doesn't know about pd.DataFrame
    df += 1  # Numba doesn't understand what this is
    return df.cov()  # or this!

In [61]:
%timeit use_pandas(x)

1.1 ms ± 118 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [62]:
@jit
def use_pandas(a):  # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a)  # Numba doesn't know about pd.DataFrame
    df += 1  # Numba doesn't understand what this is
    return df.cov()  # or this!

In [63]:
%timeit use_pandas(x)

<ipython-input-62-5550f1048843>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "use_pandas" failed type inference due to: non-precise type pyobject
During: typing of argument at <ipython-input-62-5550f1048843> (3)

File "<ipython-input-62-5550f1048843>", line 3:
def use_pandas(a):  # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a)  # Numba doesn't know about pd.DataFrame
    ^

  @jit
/home/pablo/miniconda3/lib/python3.8/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "use_pandas" was compiled in object mode without forceobj=True.

File "<ipython-input-62-5550f1048843>", line 2:
@jit
def use_pandas(a):  # Function will not benefit from Numba jit
^

  warnings.warn(errors.NumbaWarning(warn_msg,
/home/pablo/miniconda3/lib/python3.8/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compila

1.19 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


---

## Paralelismo

El paralelismo se basa en el uso de múltiples unidades de computo de manera simulánea, con el el fin de mejorar la eficiencia en rutinas de código. La idea principal consite en enfrentar un problema de programación, dividiendolo en subunidades independientes y utilizar los núcleos disponibles de la máquina para resolver tales subunidades en paralelo.

<img src='./resources/paralelo_vs_secuencial.jpeg'/>
<center>
Fuente: 
<a href='https://towardsdatascience.com/an-intro-to-parallel-computing-with-ray-d8503629485'>https://towardsdatascience.com/an-intro-to-parallel-computing-with-ray-d8503629485</a>
    
</center>



---

### Problemas Perfectamente Paralelizables o Data Parallel

La idea detrás de que se les denomine **Data Parallel** es que se aplica una función en particular sobre todos los datos (por ejemplo, multiplicar una matriz por un escalar).



Noten que la función es exactamente la misma y el calculo de esta es independiente de todas las otras funciones. Por lo mismo, estas tareas también son denominadas **perfectamente paralelizables**. 

Las operaciones elemento por elemento sobre arreglos poseen esta propiedad. 


<center>
<img src='./resources/cpu_gpu.jpg' width=500 />
</center>

<center>
Imaginense la cantidad de operaciones simples que una GPU puede lograr hacer en paralelo. Por ejemplo, sumar una matriz con otra elemento a elemento.
Fuente:    
<a href='https://www.nvidia.com/es-la/drivers/what-is-gpu-computing/'>nvidia.</a>
</center>



---

### Problemas Task Parallel


Por lo general, las subunidades de un programa no son completamente independientes y necesitan compartir información, en estos casos,se debe tener en cuenta que la comunicación entre subunidades y los datos compartidos **quitan eficiencia** al problema que se resuelve, pues se incurre en *costos de comunicación*. 


<center>
<img src='./resources/paralelismo_memoria.png' width=500 />
</center>

<center>
Fuente:    
<a href='https://manningbooks.medium.com/explaining-mapreduce-with-ducks-f643c78e0b40'>https://manningbooks.medium.com/explaining-mapreduce-with-ducks-f643c78e0b40</a>
</center>



La comunicación entre procesos es inherentemente costosa y puede llevar fallas de correctitud. Por lo general, se enfrenta el problema de costo de comunicación y correctud del manejo de memoria por medio de sistemas que se comunican por medio de **threads/hilos con memoria compartida** y **procesos con memoria distribuida**.

---

### Hilos de Procesamiento o Threads

En el caso de memoria compartida, las subunidades involucradas en el programa tienen acceso a un espacio común de memoria, este por lo general es de acceso rápido, si bien esto solventa el problema de velocidad de comunicación, el problema de correctitud sigue latente, por lo que se hace necesario utilizar técnicas de **sincronización**. 

La manera usual en la que se implementan procesos de memoria compartida es por medio de **threads** o *hilos*. Estos consisten en subtareas originadas de un proceso en particular y que comparten recursos. 


<center>
<img src='./resources/threads.jpg' width=500/>
</center>

<center>
Fuente:
<a href='https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/4_Threads.html'> https://www.cs.uic.edu/~jbell/CourseNotes/OperatingSystems/4_Threads.html </a>
</center>

---

### Procesos

Por otra parte, el concepto de memoria distribuida concibe cada subunidad como un proceso completamente separado del resto con su propio espacio de memoria asociado. En este caso, la comunicación entre procesos se debe manejar de manera explicita y es más costosa que en el caso de memoria compartida, sin embargo, se reduce el riesgo de generar errores en el manejo de memoria. 

Este tipo de paralelismos puede ser observadas en los distintos procesos que ejecuta nuestro computador.

<center>
<img src='./resources/thread_process.png' width=500/>
</center>


<center>
    Fuente:
    <a href='https://www.javamex.com/tutorials/threads/how_threads_work.shtml'>https://www.javamex.com/tutorials/threads/how_threads_work.shtml<a/>
</center>

    



---

### Threads en Python

Python puede manejar threads pero dado el diseño de su interprete, por defecto, se puede ejecutar solo una tarea a la vez, esto se conoce como **GIL** (Global Interpreter Lock). GIL provoca que cada vez que un hilo ejecute una orden de Python, se genere un bloqueo que solo será liberado una vez la ejecución del hilo termine. Esto hace que los hilos solo puedan ser ejecutados de manera secuencial.

Aunque GIL evita la ejecución paralela de las intrucciones de Python, es posible utilizar hilos mediante algunas librerías. La principal es `multiprocessing`

Multiprocessing ofrece una interfaz sencilla que incluye múltiples herramientas para manejar sincronozación y ejecución de tareas. Es posible importar esta librería de manera estándar. 

```python
import multiprocessing
```

Es posible crear procesos independientes por medio la clase `Process`, para ello basta extender el método `__init__` para inicializar los datos a procesar y generar el método `run` sobre el cual se ejecuta el proceso.

**Ejemplo**
 
Se genera un proceso independiente utilizando la clase `Process`

In [66]:
import time
from multiprocessing import Process


class Proceso_ind(Process):
    def __init__(self, num):
        super().__init__()
        self.num = num

    def run(self):
        print("Mi número:", self.num, "\nMe voy a dormir 10s 💤😴💤")
        time.sleep(10)
        print("Dormí y desperte 😃")

Para utilizar el proceso se instancia un objeto de la clase `Proceso_ind` y se llama el método `.start()` 

In [79]:
proc = Proceso_ind(5)
proc.start()

Mi número: 5 
Me voy a dormir 10s 💤😴💤
Domrí y desperte 😃


In [77]:
print("¿¿¿🤨??? Me puedo ejecutar sin esperar a que la celda anterior termine")

¿¿¿🤨??? Me puedo ejecutar sin esperar a que la celda anterior termine


**Obs**:En el ejemplo anterior, no fue necesario utilizar el metodo anulado `.run()`, este es llamado por `.start()` de manera interna.

En el caso en que se requiera esperar la finalización de un conjunto de tareas paralelas para luego recopilar resultados, es posible utilizar el método `.join()`.

In [81]:
proc = Proceso_ind(5)
proc.start()
proc.join()

print("Aquí tuve que esperar 😔")

Mi número: 5 
Me voy a dormir 10s 💤😴💤
Domrí y desperte 😃
Aquí tuve que esperar 😔


Con la construcción actual, es posible levantar tantos procesos como se requiera, en esta caso se levantan 3 procesos.

In [82]:
import time
from multiprocessing import Process


class Proceso_ind(Process):
    def __init__(self, num):
        super().__init__()
        self.num = num

    def run(self):
        print("Mi número:", self.num, "\nMe voy a dormir 3s 💤😴💤")
        time.sleep(3)
        print("Dormí y desperte 😃")

In [83]:
# Se definen los 3 procesos
proc = (Proceso_ind(1), Proceso_ind(2), Proceso_ind(3))

# Se mide el tiempo de ejecucion
start = time.time()

[*map(lambda p: p.start(), proc)]
[p.join() for p in proc]

end = time.time()


print("Tiempo de ejecución: ", end - start)

Mi número: 1 
Me voy a dormir 3s 💤😴💤
Mi número: 2 
Me voy a dormir 3s 💤😴💤
Mi número: 3 
Me voy a dormir 3s 💤😴💤
Domrí y desperte 😃
Domrí y desperte 😃
Domrí y desperte 😃
Tiempo de ejecución:  3.187493085861206


Estos tres procesos corren de manera paralela, pues su tiempo de ejecución total es aproximado al tiempo de ejecución individual. 

Es necesario comprender que el orden de ejecución de procesos paralelos no es necesariamente ordenando y predecible pues depende de cómo el sistema operativo asigne los recursos. 

---

### Pool

El módulo `multiprocessing` ofrece la clase `Pool`, esta permite manejar de manera sencilla un conjunto de procesos paralelos. Esta clase genera un conjunto de procesos llamados **workers** a los cuales se les asignan tareas por medio de los métodos `.apply()`, `.apply_async()`, `map` o `map_async`. 

El método `Pool.map()` actua de manera análoga la función nativa `map` de Python. Como resultado entrega una lista con los resultados, donde cada componente es el resultado de un worker de la clas.

**Ejemplo**

Para utilizar el método `.map` de la clase `Pool` se inicializa la clase, es posible hacerlo sin entregar un número de procesos asociados. Se genera también la función a paralelizar.

In [84]:
from multiprocessing import Pool


def func(x):
    return x ** 2 - 1


p = Pool(3)  # tambien funciona Pool()

Se comprueban los resultados y se cierra el conjunto de procesos por medio de `.close()`

In [85]:
var = [2, 4, 6, 8, 10, 12]

out = p.map(func, var)
p.close()

out

[3, 15, 35, 63, 99, 143]

El método `.map_async()` es análogo al método `.map()` con la salvedad de que retorna un objeto tipo `AsyncResult`. Esto significa que el resultado de ejecutar `.map_async()` se obtiene de manera inmediata, pudiendo continuar con las demás ordenes que proceden pero seguirá calculandose como proceso de fondo. para acceder a los resultados asociados al objeto `AsyncResult` se utiliza el método `.get()`.

Utilizamos el método de mapeo asincrónico

In [27]:
p = Pool(3)  # tambien funciona Pool()

var = [2, 4, 6, 8, 10, 12]
out = p.map_async(func, var)
out

accedemos a sus resultados

In [28]:
print(out.get())
p.close()

[3, 15, 35, 63, 99, 143]


**Ejercicios**

Los métodos `.apply()` y `apply_async()` son similares a `.map()` y `.map_async()`

1. ¿En qué se diferencian?
2. Programe una rutina que haga uso de `.apply()` y `apply_async()`. 

3. ¿Cómo se relaciona la clase `Pool` y los métodos de aplicación (`.map()`, `.apply()`, ...) con las funciones de Cython ?

### Memoria Compartida y Dataraces

Un data race es una situación que ocurre cuando uno o más hilos acceden concurrentemente a una posición de memoria o variable, al menos uno está escribiendo y al menos uno no está sincronizado con los otros hilos.

<center>
<img src='./resources/datarace_1.png' />
</center>

<center>
    Ejecución secuencial en memoria compartida por threads.
    Fuente: <a href='https://en.wikipedia.org/wiki/Race_condition'>Wikipedia</a>
</center>

<br>
<br>

<center>
<img src='./resources/datarace_2.png' />
</center>

<center>
    Ejecución paralela en memoria compartida por threads.
    Fuente: <a href='https://en.wikipedia.org/wiki/Race_condition'>Wikipedia</a>
</center>


**La solución es tener mecanismos de sincronización** de hilos. 

### Ejemplo en `multiprocessing`


El comportamiento predeterminado de `multiprocessing` es generar procesos con memoria independiente, sin embargo, permite definir ciertas variables en memoria compartida. Para definir una variable en memoria compartida se utiliza la clase `Value`, a esta clase se le entrega un tipo de dato que puede ser `i` para entero, `f` para flotante, `d` para doble precisión entre otros. 


In [29]:
from multiprocessing import Value

comp_var = Value("d")
comp_var = 55

Al utilizar variables en memoria compartida se deben tener en cuenta los procesos que acceden a ella, manejando la *concurrencia*, es decir, si los procesos pueden acceder a dichas variables de manera simultanea u ordenada. Por lo general en la actualización de valores unidimensionales se debe tener en cuenta la concurrencia bloqueando el acceso simultaneo. En arreglos se puede permitir tal manipulación siempre que los computos sean independientes. 

Para bloquear el acceso a una variable compartida se hace uso de la clase `Lock`.

In [30]:
from multiprocessing import Lock

lock = Lock()

A continuación se genera una rutina que accede a una variable de memoria compartida

In [31]:
from multiprocessing import Process, Value


class Process_shared(Process):
    def __init__(self, var, n=10000):
        super().__init__()
        self.var = var
        self.n = n

    def run(self):
        for i in range(self.n):
            self.var.value += 1

El proceso asociado toma un valor y le añade 1 hasta `n = 10000` veces por proceso. Se crea el valor inicial y se inicializan 3 procesos

In [32]:
def test():
    var = Value("i")
    var.value = 0

    procs = [Process_shared(var) for i in range(3)]

    [p.start() for p in procs]
    [p.join() for p in procs]

    print(var.value)

Se prueba el resultado

In [33]:
test()

30000


Como se puede ver, el resultado no es necesariamente 30.000, esto se debe al acceso simultaneo y aleatorio de los procesos a `var`, para solucionar este problema se hace uso de `lock`, para ello se redefine la clase `Process_shared` observando que lock es un *context manager*

In [34]:
class Process_shared_lock(Process):
    def __init__(self, var, n=10000):
        super().__init__()
        self.var = var
        self.n = n

    def run(self):
        for i in range(self.n):
            with lock:
                self.var.value += 1

Se redefine la prueba asociada y se ejecuta:

In [35]:
def test():
    var = Value("i")
    var.value = 0

    procs = [Process_shared_lock(var) for i in range(3)]

    [p.start() for p in procs]
    [p.join() for p in procs]

    print(var.value)


test()

30000


Con lo cual se obtiene el resultado buscado

---

### Paralelización con `Joblib`

Otra forma de paralelizar de forma relativamente sencilla es usar la librería `joblib`. 
Esta permite ejecutar funciones de forma paralela similar a un map. Es decir, le entregamos una lista de argumentos y ejecuta una función con dichos argumentos de forma paralela.

Para esto, utiliza el decorador `delayed` sobre una función (lo que la transforma a lazy, es decir, no se ejecuta instantaneamente). Luego a través del objeto `Parallel` que toma el número de trabajos concurrentes que se ejecutarán (`n_jobs`) ejecuta las funciones con sus parámetros.

In [86]:
(delayed(np.cos)(i) for i in np.arange(0, 1, 0.1))

<generator object <genexpr> at 0x7fb947766dd0>

In [87]:
from math import sqrt

from joblib import Parallel, delayed

Parallel(n_jobs=-1)(delayed(np.cos)(i) for i in np.arange(0, 1, 0.1))

[1.0,
 0.9950041652780258,
 0.9800665778412416,
 0.955336489125606,
 0.9210609940028851,
 0.8775825618903728,
 0.8253356149096782,
 0.7648421872844884,
 0.6967067093471654,
 0.6216099682706644]

In [88]:
%timeit [np.cos(i) for i in np.arange(0, 1, 0.001)]

1.6 ms ± 209 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [89]:
%timeit Parallel(n_jobs=-1)(delayed(np.cos)(i) for i in np.arange(0, 1, 0.001))

168 ms ± 56.8 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Quizas para tareas numéricas no es tan efectivo, para tareas pesadas, se comporta bastante bien.

Para este ejemplo, leeremos un archivo con números aleatorios en forma secuencial y en forma paralelizada:

In [39]:
import pandas as pd


def leer_archivo(_):
    _ = pd.read_csv("./resources/num_aleatorios.csv")

In [90]:
%timeit [leer_archivo(_) for _ in range(0, 20)]

1.13 s ± 48.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
%timeit Parallel(n_jobs=-1)(delayed(leer_archivo)(_) for _ in range(0,20))

440 ms ± 42.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Y ahora si notamos diferencias.

## Procesamiento Distribuido

<img src='./resources/distributed.png'/>

El procesamiento distribuido hace referencia a la ejecución de tareas utilizando múltiples máquinas. Por lo general se refiere al trabajo con clusters de procesamiento y suele llevarse a cabo por medio de herramientas como MPI. 

En Python existen diversas librerías que permiten computación distribuida. En esta última sección estudiaremos una de ellas: `Dask`.



### `Dask`


Dask permite escalar objetos y procedimientos de Python ya sea en un computador personal o un cluster de manera sencilla. Provee de funcionalidades para tratar, por medio de procesamiento multi-core, con datsets masivos **que por lo general no caben en memoria.**

**Nota**: Si tu dataset cabe en memoria comodamente, entonces quizas no es necesario usar `Dask`.

<img src='./resources/dask.png' />


Dask proporciona planificadores de bajo nivel, cuya función es sincronizar tareas entre múltiples procesos o máquinas, análogo a la librería `multiprocessing` recientemente estudiada. 


<center>
<img src='./resources/dask_mimic.png' width=700>
</center>

Pueden encontrar mayor información en la página oficial del proyecto:

https://docs.dask.org/en/latest/

### Asincronía y Corrutinas

<center>
<img src='./resources/corrutinas.png' />
</center>

Sintáxis básica de el código asincrónico

In [42]:
import asyncio


async def main():
    print("Hello ...")
    await asyncio.sleep(10)
    print("... World!")


await main()

Hello ...
... World!
